In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [2]:
import wandb
project_name = "object_detection"
wandb.init(project=project_name, reinit=True)

wandb.run.name = 'cascade_swin_fold1.ipynb'
# generted run ID로 하고 싶다면 다음과 같이 쓴다.
# wandb.run.name = wandb.run.id
wandb.run.save()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jongeson (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_1x_coco.py')

PREFIX = '../../input/data/'


In [4]:
print(cfg.dump())

model = dict(
    type='CascadeRCNN',
    pretrained=None,
    backbone=dict(
        type='SwinTransformer',
        embed_dim=96,
        depths=[2, 2, 6, 2],
        num_heads=[3, 6, 12, 24],
        window_size=7,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.0,
        ape=False,
        patch_norm=True,
        out_indices=(0, 1, 2, 3),
        use_checkpoint=False),
    neck=dict(
        type='FPN',
        in_channels=[96, 192, 384, 768],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0

In [5]:
cfg.log_config.hooks[1].init_kwargs.project = project_name
cfg.log_config.hooks[1].init_kwargs.name = wandb.run.name


In [6]:
##### albu_augmentation
albu_train_transforms = [
    dict(
        type='Resize',
        width=512,
        height=512,
        p=1.0),
    dict(
        type='Rotate',
        limit=10,
        p=0.5), 
    dict(
        type='RandomBrightnessContrast',
        p=1.0),
]
#####

In [7]:
# img_norm_cfg
# img_norm_cfg = dict(
#     mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)
# cfg.data.train.pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
#     dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
#     dict(type='Pad', size_divisor=32),
#     dict(
#         type='Albu',
#         transforms=albu_train_transforms,
#         bbox_params=dict(
#             type='BboxParams',
#             format='pascal_voc',
#             label_fields=['gt_labels'],
#             min_visibility=0.0,
#             filter_lost_elements=True),
#         keymap={
#             'img': 'image',
#             'gt_bboxes': 'bboxes'
#         },
#         update_pad_shape=False,
#         skip_img_without_anno=True),
#     dict(type='Normalize', **img_norm_cfg),
#     dict(type='DefaultFormatBundle'),
#     dict(
#         type='Collect',
#         keys=['img', 'gt_bboxes', 'gt_labels'],
#         meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
#                    'pad_shape', 'scale_factor')
#     )
# ]

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)


cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)


# num_workers?
cfg.data.samples_per_gpu = 4

# seed
cfg.seed=2020

# ??
cfg.gpu_ids = [0]

# 새롭게 만든 모델을 저장할 장소
cfg.work_dir = './work_dirs/cascade_mask_rcnn_swin_1x_trash'

# class 개수 11개로 수정
cfg.model.roi_head.bbox_head[0].num_classes=11
cfg.model.roi_head.bbox_head[1].num_classes=11
cfg.model.roi_head.bbox_head[2].num_classes=11
cfg.model.roi_head.mask_head.num_classes =11

# ??
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [8]:
model = build_detector(cfg.model)

In [9]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...


TypeError: __init__() missing 2 required positional arguments: 'doc' and 'pos'

In [10]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-19 05:26:53,238 - mmdet - INFO - load checkpoint from /opt/ml/code/Swin-Transformer-Object-Detection/cascade_mask_rcnn_swin_tiny_patch4_window7_1x.pth
2021-05-19 05:26:53,242 - mmdet - INFO - Use load_from_local loader
Done (t=0.86s)
creating index...
index created!
2021-05-19 05:26:53,537 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.0.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.0.fc_reg.bias: copying a param with shape t

2021-05-19 05:28:27,994 - mmdet - INFO - Epoch [1][50/655]	lr: 9.890e-06, eta: 11:07:40, time: 1.703, data_time: 0.090, memory: 15677, loss_rpn_cls: 0.2126, loss_rpn_bbox: 0.0378, s0.loss_cls: 2.3447, s0.acc: 15.6680, s0.loss_bbox: 0.4631, s0.loss_mask: 1.7185, s1.loss_cls: 0.9513, s1.acc: 43.7471, s1.loss_bbox: 0.0856, s1.loss_mask: 0.8557, s2.loss_cls: 0.6565, s2.acc: 3.9150, s2.loss_bbox: 0.0117, s2.loss_mask: 0.4553, loss: 7.7927, grad_norm: 41.1062


KeyboardInterrupt: 